In [1]:
from science_jubilee.Machine import Machine
from science_jubilee.tools.Camera import Camera
from science_jubilee.decks.Deck import Deck
%load_ext autoreload
%autoreload 2

In [2]:
m = Machine(address='jubilee.local')

/home/pi/science_jubilee/science_jubilee/Machine.py:332: UserWarning: Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.
  warnings.warn("Connection initiated with tool equipped. Use reload_tool() after instantiate this tool.")


In [3]:
camera = Camera(3, "camtool")

In [4]:
m.reload_tool(camera)

In [7]:
m.pickup_tool(camera)

In [6]:
deck = m.load_deck("lab_automation_deck_MAfeb")

In [8]:
# l = deck.load_labware("corning_24_wellplate_3.4ml_flat", 0)
l = deck.load_labware("corning_96_wellplate_360ul_flat", 2)

In [9]:
well = l["A1"]
well

Well(name='A1', depth=10.67, totalLiquidVolume=360, shape='circular', diameter=6.86, xDimension=None, yDimension=None, x=33.88, y=176.83999999999997, z=3.55, offset=[19.5, 102.6], slot=2)

In [ ]:
m.move_to(x=well.x, y=well.y)

In [54]:
f = camera.get_well_image(well=well)

In [55]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def process_image(image_bin):
    """
    externally callable function to run processing pipeline
    
    takes an image as a bstring
    """
    image = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
    radius = 30
    masked_image = _mask_image(image, radius)
    plt.imshow(masked_image)
    plt.show()
    values = _get_rgb_avg(masked_image)
    return values



def _mask_image(image, radius):

    mask = np.zeros(image.shape[:2], dtype = "uint8")
    cv2.circle(mask, (600, 600), radius, 255, -1)
    masked = cv2.bitwise_and(image, image, mask=mask)

    return masked



def _get_rgb_avg(image):
    bgr = []
    for dim in [0,1,2]:
        flatdim = image[:,:,dim].flatten()
        indices = flatdim.nonzero()[0]
        value = flatdim.flatten()[indices].mean()
        bgr.append(value)

    rgb = [bgr[i] for i in [2,1,0]]

    return rgb

In [56]:
RGB = process_image(f)

In [57]:
RGB

[81.42923351158645, 41.96738745125842, 124.6937256292095]